# Neural Network

In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Dense
import numpy as np

# place holder data
X_train = np.random.randn(50, 256,256, 3)
y_train = np.round(np.random.randn(50, 256,256, 3))
input_shape = (256, 256, 3)

network = keras.Sequential()
network.add(keras.Input(input_shape))
network.add(Dense(12, activation='linear'))
network.add(Dense(8, activation='relu'))
network.add(Dense(3, activation='sigmoid'))
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256, 256, 12)      48        
_________________________________________________________________
dense_1 (Dense)              (None, 256, 256, 8)       104       
_________________________________________________________________
dense_2 (Dense)              (None, 256, 256, 3)       27        
Total params: 179
Trainable params: 179
Non-trainable params: 0
_________________________________________________________________


In [2]:
network.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=True))

network.fit(X_train, y_train, batch_size=12, epochs=2)

Train on 50 samples
Epoch 1/2
50/50 [==============================] - 87s 2s/sample - loss: 0.9805
Epoch 2/2
50/50 [==============================] - 0s 3ms/sample - loss: 0.9748


In [3]:
y_test = network.predict(np.random.randn(20,256,256,3))
print(y_test.shape)

(20, 256, 256, 3)
